In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about Inception?",
        "answer": """
Here is what I know:
Director: Christopher Nolan
Main Cast: Leonardo DiCaprio, Joseph Gordon‑Levitt, Ellen Page, Tom Hardy
Budget: ≈ $160 million
Box Office: ≈ $836 million worldwide
Genre: Science‑fiction / Heist / Thriller
Synopsis: A master thief who invades people’s dreams to steal secrets is hired to perform the inverse—plant an idea—forcing his team to navigate multiple layers of mind‑bending reality.
""",
    },
    {
        "question": "What do you know about Titanic?",
        "answer": """
Here is what I know:
Director: James Cameron
Main Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
Budget: ≈ $200 million
Box Office: ≈ $2.2 billion worldwide
Genre: Epic Romance / Historical Drama
Synopsis: A blossoming romance between an aristocrat and a penniless artist unfolds aboard the ill‑fated RMS Titanic, culminating in tragedy when the “unsinkable” ship hits an iceberg.
""",
    },
    {
        "question": "What do you know about Parasite?",
        "answer": """
Here is what I know:
Director: Bong Joon‑ho
Main Cast: Song Kang‑ho, Lee Sun‑kyun, Cho Yeo‑jeong, Choi Woo‑shik, Park So‑dam
Budget: ≈ $11 million
Box Office: ≈ $258 million worldwide
Genre: Dark Comedy / Thriller / Social Satire
Synopsis: A resource‑poor family cons its way into the lives of a wealthy household, setting off an escalating chain of deception, class tension, and shocking consequences.
""",
    },
]

chat.predict("What do you know about Frozen?")


example_template="""
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix= "Human: What do you know about {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({
    'movie': 'frozen'
})

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What do you know about Frozen?"
  ]
}
Frozen is a popular animated film produced by Walt Disney Animation Studios. It was released in 2013 and is based on the Hans Christian Andersen fairy tale "The Snow Queen." The film follows the story of two sisters, Elsa and Anna, who live in the kingdom of Arendelle. Elsa has magical ice powers that she struggles to control, leading her to accidentally plunge the kingdom into eternal winter. Anna sets out on a journey to find her sister and save Arendelle with the help of an ice harvester named Kristoff, his loyal reindeer Sven, and a comical snowman named Olaf. Frozen was a critical and commercial success, winning two Academy Awards for Best Animated Feature and Best Original Song for "Let It Go." It spawned a sequel, Frozen II, which was released in 2019. The franchise also includes merchandise, theme park attractions, and a Broadway musical adaptation.[llm

AIMessageChunk(content='AI: \nHere is what I know:\nTitle: Frozen\nDirectors: Chris Buck, Jennifer Lee\nMain Cast: Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad\nBudget: ≈ $150 million\nBox Office: ≈ $1.28 billion worldwide\nGenre: Animated Musical / Fantasy\nSynopsis: Two sisters, Princess Anna and Queen Elsa, navigate their relationship and the magical powers of Elsa, which inadvertently plunges their kingdom into eternal winter.')